<a href="https://colab.research.google.com/github/edwin-yan/Passion-Fruit-Disease-Detection/blob/andrew/Image_Augmentation_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image Augmentation

Using this library to do image augmentation
https://towardsdatascience.com/tensorflow-and-image-augmentation-3610c6c243a2

# SET UP
## Mounting

Note that this assumes that your google drive folder has the same document layout. You must modify if that is not the case.


From your root Google Drive directory create a couple of folders:
* `JHU/DNN/FinalProject/Train_Images`
* `JHU/DNN/FinalProject/Test_Images` 

with Train and test images unzipped in those directories



# **TODO:** GLOBAL VARIABLE set to `True` if using colab and `False` if not!

In [1]:
IS_COLAB = True 

# **TODO:** CHANGE `IMG_DIR` if your google drive directory is different

In [2]:
if IS_COLAB:
  from google.colab import drive
  drive.mount('/content/drive')
  IMG_PATH = '/content/drive/MyDrive/JHU/DNN/FinalProject'
else:
  IMG_PATH= '.'

Mounted at /content/drive


In [3]:
TRAIN_PATH = f'{IMG_PATH}/Train.csv'
TEST_PATH = f'{IMG_PATH}/Test.csv'

In [4]:
import time

t0 = time.time()

## Imports

In [5]:
import tensorflow as tf

Import tf addons
For some reason colab does not allow minor version e.g. specifying 2.6

In [6]:
!pip -q install -U tensorflow_addons > log 
import tensorflow_addons as tfa

In [7]:
!pip install tf-image
from tf_image.core.bboxes.rotate import random_rotate
from tf_image.core.random import random_function_bboxes
from tf_image.core.bboxes.flip import flip_left_right

In [ ]:
# Clone from repo and cd into repo
!git clone https://github.com/edwin-yan/Passion-Fruit-Disease-Detection

# Change directory
%cd Passion-Fruit-Disease-Detection

# Checkout specific branch
!git checkout andrew

Cloning into 'Passion-Fruit-Disease-Detection'...
remote: Enumerating objects: 348, done.
remote: Counting objects: 100% (348/348), done.


In [ ]:
!ls

In [ ]:
import pandas as pd
from helper import plot_sample_images, plot_sample_images_multi

In [ ]:
import matplotlib.image as img
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
# https://docs.python.org/3/library/uuid.html
import uuid

In [ ]:
df_train, df_test = pd.read_csv(TRAIN_PATH), pd.read_csv(TEST_PATH)
print(f"Training Data Shape:  {df_train.shape}\nTest Data Shape:      {df_test.shape}")

In [ ]:
df_train.Image_ID.value_counts().head(10)

In [ ]:
df_train['class'].value_counts()

In [ ]:
# plot_sample_images_multi(df_train,dir_path=IMG_PATH)

In [ ]:
# plot_sample_images_multi(df_test, show_label=False, is_train=False,dir_path=IMG_PATH)

Try Image Augmentation

I think the require format is https://github.com/Ximilar-com/tf-image/blob/master/tf_image/core/bboxes/rotate.py

 [ymin, xmin, ymin, xmax].

## FIRST update the df_train

In [ ]:
IMG_HEIGHT = 512
IMG_WIDTH = 512

In [ ]:
df_train.head()

Copy for use later

In [ ]:
df_original = df_train.copy()

In [ ]:
# get xmax
df_train['xmax'] = df_train['xmin'] + df_train['width']
# get ymax
df_train['ymax'] = df_train['ymin'] + df_train['height']

In [ ]:
df_train

In [ ]:
imgID = 'ID_007FAIEI'

In [ ]:
def flipLR(bounding_boxes):
  bboxes = bounding_boxes * tf.constant([1, -1, 1, -1], dtype=tf.float32) + tf.stack([0.0, 1.0, 0.0, 1.0])
  bboxes = tf.stack([bboxes[:, 0], bboxes[:, 3], bboxes[:, 2], bboxes[:, 1]], axis=1)
  return bboxes

In [ ]:
def rotateImage(imgID,imgClass, xmin_toRot, ymin_toRot,xmax_toRot, ymax_toRot ,df_original):
  train_toRotate_path = f'{IMG_PATH}/Train_Images/{imgID}.jpg'
  image = img.imread(train_toRotate_path)

  xmin_toRot = xmin_toRot
  ymin_toRot = ymin_toRot
  xmax_toRot = xmax_toRot
  ymax_toRot = ymax_toRot

  # 
  # fig, ax = plt.subplots()
  # from matplotlib.patches import Rectangle
  # ax.imshow(image)
  # rect = Rectangle((xmin_toRot,ymin_toRot),xmax_toRot-xmin_toRot,ymax_toRot-ymin_toRot, linewidth=1, edgecolor='r', facecolor='none')

  # ax.add_patch(rect)
  # plt.show()

  image = tf.io.read_file(train_toRotate_path)
  image = tf.image.decode_jpeg(image)
  bounding_boxes = tf.constant([[ymin_toRot,xmin_toRot,  ymax_toRot,xmax_toRot ]],dtype=tf.float32)
  image_rot, bb  = flip_left_right(
      image, bounding_boxes
  )

  bboxes = flipLR(bounding_boxes)
  rotatedBB = (np.array(bboxes))[0]
  rotYmin = rotatedBB[0]
  rotXmin = rotatedBB[1] + 512
  rotYmax = rotatedBB[2]
  rotXmax = rotatedBB[3] + 512
  width = rotXmax - rotXmin
  height = rotYmax - rotYmin

  #
  # fig, ax = plt.subplots()
  # from matplotlib.patches import Rectangle
  # ax.imshow(image_rot)
  # rect = Rectangle((rotXmin,rotYmin),rotXmax-rotXmin,rotYmax-rotYmin, linewidth=1, edgecolor='r', facecolor='none')

  # ax.add_patch(rect)
  # plt.show()

  # Write augmented image to folder: 
  # create uuid based on time
  new_imgID = f'{imgID}_{uuid.uuid1()}'
  tf_image = tf.image.encode_jpeg(image_rot)
  tf.io.write_file(f'{IMG_PATH}/Train_Images_Augmented/{new_imgID}.jpg', tf_image)

  df_original = df_original.append({  'Image_ID':new_imgID,	'class':imgClass,	'xmin':rotXmin,	'ymin':rotYmin,	'width':width,	'height':height},ignore_index=True)

  return df_original, image_rot, rotXmin,  rotYmin, width, height


In [ ]:
  xmin_toRot = df_train.loc[df_train['Image_ID'] == imgID]['xmin'].values[0]
  ymin_toRot = df_train.loc[df_train['Image_ID'] == imgID]['ymin'].values[0]
  xmax_toRot = df_train.loc[df_train['Image_ID'] == imgID]['xmax'].values[0]
  ymax_toRot = df_train.loc[df_train['Image_ID'] == imgID]['ymax'].values[0]

In [ ]:
df_original, image_rot,  rotXmin,  rotYmin, width, height = rotateImage(imgID,'fruit_woodiness', xmin_toRot, ymin_toRot,xmax_toRot, ymax_toRot,df_original )

https://stackoverflow.com/questions/55737105/why-wont-tf-write-file-write-a-file

In [ ]:
for index, row in df_train.iterrows():
    df_original, image_rot,  rotXmin,  rotYmin, width, height  = rotateImage(row['Image_ID'], row['xmin'], row['ymin'], row['xmax'],row['ymax'])

https://stackoverflow.com/questions/54255431/invalidargumenterror-cannot-compute-matmul-as-input-0zero-based-was-expected

https://stackoverflow.com/questions/37435369/matplotlib-how-to-draw-a-rectangle-on-image

In [ ]:
df_original.to_csv(f'{IMG_PATH}/train_augmented.csv', encoding='utf-8', index=False)